<a href="https://colab.research.google.com/github/nangokosu/ebay_ml/blob/main/DenseNet_Siamese_Cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.layers import *
import numpy as np
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import color,data,transform,filters,util,restoration
import tensorflow as tf

In [ ]:
from keras.models import Model

In [ ]:
import tensorflow.keras.backend as K

def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)
    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)
    x = Multiply()([init, se])
    return x

In [ ]:
def create_encoder(input_shape):
  inputs=Input(shape=input_shape)
  resnet = keras.applications.DenseNet169(
        include_top=False, weights='imagenet', input_tensor=inputs, pooling=None
    )
  resnet.trainable=False
  outputs=squeeze_excite_block(resnet.output,ratio=16)
  outputs=GlobalAveragePooling2D()(outputs)
  outputs=Dropout(0.2)(outputs)
  outputs=Dense(120,activation="relu")(outputs)
  outputs=Dense(50,use_bias=False)(outputs)
  #outputs=BatchNormalization()(outputs)
  model = Model(
      inputs=inputs, outputs=outputs, name='encoder'
    )
  return model

In [ ]:
input_shape=(32,32,3)
encoder=create_encoder(input_shape)
input_1=Input(shape=(32,32,3))
input_2=Input(shape=(32,32,3))

51879936/51877672 [==============================] - 2s 0us/step


In [ ]:
# ONLY APPLIES FOR RETRAINING
# encoder=tf.keras.load_model("/content/drive/Shareddrives/eBay ML/DenseNet__siamese_Encoder)

In [ ]:
encoded_1=encoder(input_1)
encoded_2=encoder(input_2)

In [ ]:
def cosine_distance(encoded_no1,encoded_no2):
  predicted_sim=Dot(axes=1,normalize=True)([encoded_1,encoded_2])
  return predicted_sim

In [ ]:
predicted_sim=cosine_distance(encoded_1,encoded_2)
sim_model=Model([input_1,input_2],predicted_sim)

In [ ]:
sim_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
encoder (Functional)            (None, 50)           13194792    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dot (Dot)                       (None, 1)            0           encoder[0][0]                

In [ ]:
lrplateau1=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.20,patience=8)

In [ ]:
modelCheckpoint=tf.keras.callbacks.ModelCheckpoint('/content/drive/Shareddrives/eBay ML/DenseNet_Res_Cosine',save_best_only=True)
# might be saved in own drive right now, check to be sure

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [ ]:
sim_model.compile(optimizer="Adam",loss="mean_squared_error")

In [ ]:
from keras.applications.densenet import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_pickle("/content/drive/Shareddrives/eBay ML/normalized_df_pairs_balanced.pkl")
length = df.shape[0]
df = df.iloc[2*length // 4: 3 * length // 4]
#df.head()
##df["image"]=df.input_tup.apply(lambda x: x[0])
#df['label']=df.input_tup.apply(lambda x: x[1])
#labels=pd.get_dummies(df.label)
# -1 is not similar

In [ ]:
image_1=preprocess_input(np.stack(df["image_1"]))

In [ ]:
image_2=preprocess_input(np.stack(df["image_2"]))

In [ ]:
sim_model.fit([image_1,image_2],df.similarity.values,validation_split=0.10,epochs=60,batch_size=64,callbacks=[lrplateau1,modelCheckpoint,early_stop])

Epoch 1/60
1387/1387 [==============================] - 629s 442ms/step - loss: 0.8214 - val_loss: 0.9052
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/DenseNet_Res_Cosine/assets
Epoch 2/60
1387/1387 [==============================] - 609s 439ms/step - loss: 0.7708 - val_loss: 0.9011
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/DenseNet_Res_Cosine/assets
Epoch 3/60
1387/1387 [==============================] - 614s 443ms/step - loss: 0.7543 - val_loss: 0.8993
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/DenseNet_Res_Cosine/assets
Epoch 4/60
1387/1387 [==============================] - 614s 443ms/step - loss: 0.7460 - val_loss: 0.8955
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/DenseNet_Res_Cosine/assets
Epoch 5/60
1387/1387 [==============================] - 616s 444ms/step - loss: 0.7304 - val_loss: 0.9038
Epoch 6/60
1387/1387 [==============================] - 614s 443ms/step - loss

In [ ]:
tf.keras.models.save_model(encoder,"/content/drive/Shareddrives/eBay ML/DenseNet__siamese_Encoder")

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/DenseNet__siamese_Encoder/assets
